# Instalar librerías

In [1]:
%%writefile requirements.txt
pytz
tweepy==3.9.0
loguru==0.5.3
google-cloud-pubsub==2.1.0

Writing requirements.txt


In [2]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.13.0
    Uninstalling tweepy-4.13.0:
      Successfully uninstalled tweepy-4.13.0
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.0
    Uninstalling google-api-core-2.11.0:
      Successfully uninstalled google-api-core-2.11.0


# Importar librerías

In [3]:
import sys
import os
import pandas as pd
import tweepy
import json

# Conexiones

In [4]:
PROJECT_ID = "proyectodespliguealg" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path_tweets='drive/My Drive/keepcoding/Tweets/tweets/'

In [7]:

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

In [8]:
os.environ["CONSUMER_KEY"] = "Bygk57hOHq1yDoIrOJXwniJfE"
os.environ["CONSUMER_SECRET"] = "IDhUZIY5pY6bdsPy8JPrqyusy5QBM2Faat1l5YCPzQgNcQrKiR"
os.environ["BEARER_TOKEN"] = "AAAAAAAAAAAAAAAAAAAAABvbmQEAAAAA7v%2FF97fezv%2FxTgF1oc5dNJprnvM%3DnDvsv5P9qllipgkr64oBB6h5XM6xIFyrzIzPTNymhmlcFgUUeO"

os.environ["ACCESS_TOKEN"] = "230861654-cq1hrMhEWa5rhlx6vx6I7O147IcRmR2qiRF6n7YQ"
os.environ["ACESSS_TOKEN_SECRET"] = "mSKxgXWZc8gNZWgpEZdoTzJU2rbFjuImktTkkz1zCe9qp"
os.environ["TOPIC_NAME"] = "tweets"
os.environ["PROJECT_ID"] = PROJECT_ID

In [9]:
tweepy.__version__

'3.9.0'

# Cargo archivo con categorías para exportar información

In [10]:
# importaría un csv con categorías a explorar en los tweets



# Función para Exportar tweets

https://patricrp.medium.com/buscando-contenido-con-la-api-de-twitter-f3c12994a77f

In [11]:
auth = tweepy.OAuthHandler(os.environ["CONSUMER_KEY"], os.environ["CONSUMER_SECRET"])
auth.set_access_token(os.environ["ACCESS_TOKEN"], os.environ["ACESSS_TOKEN_SECRET"])

In [12]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [13]:
def searchTweets_bytopic(query,api, lang='es', num_tweets=10,exclude_replies=True,exclude_retweets=True,result_type='popular'):
    '''
    Perform a query search request to Twitter with Tweepy. Parameters:
    - query: include a word or a complex query using operators
    - lang: language ISO code 639-1
    - geo: latitude, longitude and km or miles
    - num_tweets: more than 10
    -result_type: mixed : include both popular and real time results in the response, recent : return only the most recent results in the response or popular : return only the most popular results in the response
    
    Returns a dataframe with the columns: query, date, id, user and text.
    '''
    print('Creating an empty dataframe with the first 10 tweets')

    if exclude_replies==True:
      if exclude_retweets==True:
        query=query+' exclude:replies exclude:retweets'
      else:
        query=query+' exclude:replies'
    else:
      if exclude_retweets==True:
        query=query
      else:
        query=query+' exclude:retweets'

    print('Creating query...')
    print(query)

    try:
        count = 0
        #First request
        while count < 1:
            tweets = tweepy.Cursor(api.search,
                               q=query,
                               lang=lang,
                              #  geo=geo,
                               result_type=result_type).items(10)


            data = [[query,tweet.created_at, tweet.id, tweet.user.screen_name, tweet.text] for tweet in tweets]

            df_tw = pd.DataFrame(data=data, columns=['query','date', 'id', 'user', 'tweet_text'])
            print('Here you have your first 10 tweets')
            # print(df_tw)

            count += 1

        #Add more tweets
        while count >= 1:
            tweets = tweepy.Cursor(api.search,
                                       q=query,
                                       lang=lang,
                                      #  geo=geo,
                                       result_type=result_type,
                                       max_id=df_tw.id.min()).items(num_tweets-10)


            data = [[query, tweet.id, tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweets]
            df2 = pd.DataFrame(data=data, columns=['query', 'id', 'date', 'user', 'tweet_text'])
            df_tw = df_tw.append(df2, sort=True)
            df_tw['query']=query

            count += 1
            print('Adding ',str(num_tweets),'to the dataframe')
            print('Your dataframe has a total of', df_tw.shape[0], 'tweets.')
            print('Oldest tweet is from', df_tw.date.min())

            return df_tw
    except AttributeError:
        pass
    except NameError:
        pass

# Exportar tweets por categorias

## info para hacer mapa de palabras relacionadas con el abandono escolar

In [14]:
import time
now=time.strftime("%d_%m_%y_%H:%M:%S")


In [15]:
keys='school'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'general/'+keys+'_'+str(now)+'.csv')

Creating an empty dataframe with the first 10 tweets
Creating query...
school exclude:replies exclude:retweets
Here you have your first 10 tweets


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


Adding  1000 to the dataframe
Your dataframe has a total of 63 tweets.
Oldest tweet is from 2023-04-27 03:45:10


In [16]:
df0.head()

,date,id,query,tweet_text,user
0,2023-05-05 09:59:28,1654425569243586561,school exclude:replies exclude:retweets,"Strongly condemn the horrific, brutal killing ...",ImranKhanPTI
1,2023-05-05 12:44:44,1654467163560837121,school exclude:replies exclude:retweets,DISGUSTING: North Carolina Lt. Gov. Mark Robin...,joncoopertweets
2,2023-05-04 22:27:02,1654251313520328704,school exclude:replies exclude:retweets,The internet has taught me more than school ev...,todayyearsoldig
3,2023-05-05 16:03:30,1654517183874203648,school exclude:replies exclude:retweets,We now know that Justice Sotomayor didn't recu...,charliekirk11
4,2023-05-06 02:15:00,1654671070576902151,school exclude:replies exclude:retweets,I wish they taught us this in school instead o...,todayyearsoldig


In [17]:
keys='education'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'general/'+keys+'_'+str(now)+'.csv')

Creating an empty dataframe with the first 10 tweets
Creating query...
education exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 54 tweets.
Oldest tweet is from 2023-04-27 04:58:24


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


In [18]:
keys='lessons'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'general/'+keys+'_'+str(now)+'.csv')

Creating an empty dataframe with the first 10 tweets
Creating query...
lessons exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 44 tweets.
Oldest tweet is from 2023-04-27 06:57:33


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


In [19]:
keys='teacher'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'general/'+keys+'_'+str(now)+'.csv')

Creating an empty dataframe with the first 10 tweets
Creating query...
teacher exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 40 tweets.
Oldest tweet is from 2023-04-27 07:19:37


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


In [20]:
keys='education'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'general/'+keys+'_'+str(now)+'.csv')

Creating an empty dataframe with the first 10 tweets
Creating query...
education exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 54 tweets.
Oldest tweet is from 2023-04-27 04:58:24


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


## Tweets para hacer análisis de sentimiento por categorías

## Cluster 0

In [21]:
cluster_4tweets={0: ['digital', 'devices', 'outside', 'home', 'available'],
 1: ['text', 'cognitive', 'process', 'structure', 'understand'],
 2: ['people', 'money', 'home', 'find', 'parents'],
 3: ['language', 'money', 'acquired', 'skills', 'information'],
 4: ['mathematics', 'science']}

In [22]:
# cluster 0
keys='digital devices'
file_name='Cluster0'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
digital devices exclude:replies exclude:retweets
Here you have your first 10 tweets


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


Adding  1000 to the dataframe
Your dataframe has a total of 2 tweets.
Oldest tweet is from 2023-05-06 12:56:09


,date,id,query,tweet_text,user
0,2023-05-06 12:56:09,1654832421236342784,digital devices exclude:replies exclude:retweets,Take time this weekend to rest and reflect. St...,BebeChez_eth
0,2023-05-06 12:56:09,1654832421236342784,digital devices exclude:replies exclude:retweets,Take time this weekend to rest and reflect. St...,BebeChez_eth


In [23]:
# cluster 0
keys='digital devices school'
file_name='Cluster0'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
digital devices school exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [24]:
# cluster 0
keys='tablet school'
file_name='Cluster0'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
tablet school exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [25]:
# cluster 0
keys='smartphone school'
file_name='Cluster0'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
smartphone school exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


## Cluster 1

In [26]:
cluster_4tweets={0: ['digital', 'devices', 'outside', 'home', 'available'],
 1: ['text', 'cognitive', 'process', 'structure', 'understand'],
 2: ['people', 'money', 'home', 'find', 'parents'],
 3: ['language', 'money', 'acquired', 'skills', 'information'],
 4: ['mathematics', 'science']}

In [27]:
# cluster 1
keys='kids comprehension of texts'
file_name='Cluster1'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
kids comprehension of texts exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [28]:
# cluster 1
keys='school language'
file_name='Cluster1'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
school language exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 4 tweets.
Oldest tweet is from 2023-04-28 18:33:37


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-06 14:00:36,1654848641075388419,school language exclude:replies exclude:retweets,Until Africans use the most popular local lang...,HustleKing01
1,2023-04-28 18:33:37,1652018245006929921,school language exclude:replies exclude:retweets,Oh my god. \n\nA high school baseball game in ...,BenVerlander
2,2023-04-29 11:55:07,1652280346006659072,school language exclude:replies exclude:retweets,1. I hate libraries \n2. I hate schools\n3. Th...,3YearLetterman
0,2023-04-28 18:33:37,1652018245006929921,school language exclude:replies exclude:retweets,Oh my god. \n\nA high school baseball game in ...,BenVerlander


In [29]:
# cluster 1
keys='lesson language'
file_name='Cluster1'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
lesson language exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 2 tweets.
Oldest tweet is from 2023-05-06 06:37:31


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-06 06:37:31,1654737135226765312,lesson language exclude:replies exclude:retweets,Myra: We don’t need your sympathy and neither ...,PrinceArihan
0,2023-05-06 06:37:31,1654737135226765312,lesson language exclude:replies exclude:retweets,Myra: We don’t need your sympathy and neither ...,PrinceArihan


## Cluster 2

In [30]:
cluster_4tweets={0: ['digital', 'devices', 'outside', 'home', 'available'],
 1: ['text', 'cognitive', 'process', 'structure', 'understand'],
 2: ['people', 'money', 'home', 'find', 'parents'],
 3: ['language', 'money', 'acquired', 'skills', 'information'],
 4: ['mathematics', 'science']}

In [31]:
# cluster 2
keys='school money'
file_name='Cluster2'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
school money exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 23 tweets.
Oldest tweet is from 2023-04-27 11:53:10


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-06 00:30:01,1654644652593512448,school money exclude:replies exclude:retweets,Elon Musk came to the US with no money &amp; g...,cb_doge
1,2023-05-06 01:32:19,1654660332022013958,school money exclude:replies exclude:retweets,EMERGENCY HOMELESS MISSION #Marietta #GA\n\nAr...,codeofvets
2,2023-05-04 19:45:20,1654210620630155265,school money exclude:replies exclude:retweets,My thoughts on #HB5 or corporate welfare:\n\nT...,DonHuffines
3,2023-05-04 20:23:10,1654220142258135040,school money exclude:replies exclude:retweets,“We are done sending our money \n to Ukraine.\...,William_E_Wolfe
4,2023-05-04 11:43:20,1654089321211744259,school money exclude:replies exclude:retweets,"£250,000,000 of our money will be spent on the...",DrProudman


In [32]:
# cluster 2
keys='school money parents'
file_name='Cluster2'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
school money parents exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 4 tweets.
Oldest tweet is from 2023-04-30 18:49:33


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-01 13:37:21,1653030853407297537,school money parents exclude:replies exclude:r...,Nashville shooting update:\n\nDid you know tha...,KeenanPeachy
1,2023-05-04 11:43:22,1654089328916590606,school money parents exclude:replies exclude:r...,I’m not sure how to make this any clearer - pa...,PolitiBunny
2,2023-04-30 18:49:33,1652747032476413953,school money parents exclude:replies exclude:r...,"The man asks her, ""Where are your parents? Whe...",jenniferzeng97
0,2023-04-30 18:49:33,1652747032476413953,school money parents exclude:replies exclude:r...,"The man asks her, ""Where are your parents? Whe...",jenniferzeng97


In [33]:

# cluster 2
keys='money parents home'
file_name='Cluster2'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
money parents home exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 2 tweets.
Oldest tweet is from 2023-04-27 16:36:10


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-04-27 16:36:10,1651626299255844869,money parents home exclude:replies exclude:ret...,"On a call with a mentor last week he says, “My...",taylorawelch
0,2023-04-27 16:36:10,1651626299255844869,money parents home exclude:replies exclude:ret...,"On a call with a mentor last week he says, “My...",taylorawelch


## Cluster 3

In [34]:
cluster_4tweets={0: ['digital', 'devices', 'outside', 'home', 'available'],
 1: ['text', 'cognitive', 'process', 'structure', 'understand'],
 2: ['people', 'money', 'home', 'find', 'parents'],
 3: ['language', 'money', 'acquired', 'skills', 'information'],
 4: ['mathematics', 'science']}

In [35]:
# cluster 3
keys='school skills information'
file_name='Cluster3'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
school skills information exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [36]:
# cluster 3
keys='kids skills information'
file_name='Cluster3'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
kids skills information exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [37]:
# cluster 3
keys='skills information'
file_name='Cluster3'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
skills information exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 6 tweets.
Oldest tweet is from 2023-05-01 12:00:01


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-05 20:25:25,1654583097415925762,skills information exclude:replies exclude:ret...,Dear pro-🇷🇺 supporters. This week is going to ...,OlgaBazova
1,2023-05-03 08:13:08,1653674033739706368,skills information exclude:replies exclude:ret...,"🎥Cadets, time to dust off your cameras and sha...",Caduceus_CMP
2,2023-05-01 12:00:01,1653006354964000769,skills information exclude:replies exclude:ret...,Think critically &amp; work together - that's ...,ArmedwScience
3,2023-05-01 23:58:58,1653187284538388481,skills information exclude:replies exclude:ret...,I will be writing a detailed report at https:/...,BrianRoemmele
4,2023-05-01 13:58:51,1653036260741586948,skills information exclude:replies exclude:ret...,SKILLS TRAINING PLEASE 🙏 \n\nDribbling &amp; p...,_ToddBeane


In [38]:
# cluster 3
keys='language money'
file_name='Cluster3'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
language money exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 10 tweets.
Oldest tweet is from 2023-04-27 13:16:02


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-04 16:52:41,1654167172581556250,language money exclude:replies exclude:retweets,You won't find a better researched book exposi...,iluminatibot
1,2023-05-02 21:55:18,1653518554409443329,language money exclude:replies exclude:retweets,"""I wanna make money with you""\n- a love language",kaylaparrilla
2,2023-05-03 21:59:19,1653881953181892609,language money exclude:replies exclude:retweets,"The Federal Reserve isn't even federal, it's a...",iluminatibot
3,2023-05-02 08:27:12,1653315189096480768,language money exclude:replies exclude:retweets,Should taxpayers money be given to trolls to a...,rohini_sgh
4,2023-04-27 13:16:02,1651575933671821314,language money exclude:replies exclude:retweets,"If you want to make more money, then you need ...",The_MMW


## Cluster 4

In [39]:
cluster_4tweets={0: ['digital', 'devices', 'outside', 'home', 'available'],
 1: ['text', 'cognitive', 'process', 'structure', 'understand'],
 2: ['people', 'money', 'home', 'find', 'parents'],
 3: ['language', 'money', 'acquired', 'skills', 'information'],
 4: ['mathematics', 'science']}

In [40]:
# cluster 4
keys='school mathematics'
file_name='Cluster4'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
school mathematics exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 3 tweets.
Oldest tweet is from 2023-05-04 17:36:57


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-05 09:44:45,1654421868974145538,school mathematics exclude:replies exclude:ret...,"Azuokeke, 15, scored a total of 337 marks in t...",TheNationNews
1,2023-05-04 17:36:57,1654178310622937102,school mathematics exclude:replies exclude:ret...,"Azuokeke, 15, scored a total of 337 marks in t...",TheNationNews
0,2023-05-04 17:36:57,1654178310622937102,school mathematics exclude:replies exclude:ret...,"Azuokeke, 15, scored a total of 337 marks in t...",TheNationNews


In [41]:
# cluster 4
keys='school science'
file_name='Cluster4'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
school science exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 15 tweets.
Oldest tweet is from 2023-04-27 16:04:57


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
0,2023-05-02 03:05:14,1653234163233767424,school science exclude:replies exclude:retweets,Thomas Jefferson University forced its preside...,DrJBhattacharya
1,2023-04-28 18:17:09,1652014103815176194,school science exclude:replies exclude:retweets,This is the reading list at the public high sc...,thevivafrei
2,2023-05-06 07:52:57,1654756117946281984,school science exclude:replies exclude:retweets,Wesley Girls High School v St. Louis SHS\n\nAp...,tv3_ghana
3,2023-05-06 13:18:33,1654838060499210241,school science exclude:replies exclude:retweets,Arts - 100% Commerce - 100% \nScience - 98.6% ...,RKamatOfficial
4,2023-05-01 08:33:37,1652954414745829378,school science exclude:replies exclude:retweets,"Snapshots of the 1st ""Open Discussion on IPR p...",KIITUniversity


In [42]:
# cluster 4
keys='lessons mathematics'
file_name='Cluster4'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
lessons mathematics exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [43]:
# cluster 4
keys='teach mathematics'
file_name='Cluster4'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
teach mathematics exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user


In [44]:
# cluster 4
keys='teach science'
file_name='Cluster4'
df0=searchTweets_bytopic(keys,lang='en',num_tweets=1000,api=api)
df0.to_csv(path_tweets+'clusters/'+file_name+'_'+keys+'_'+str(now)+'.csv')
df0.head()

Creating an empty dataframe with the first 10 tweets
Creating query...
teach science exclude:replies exclude:retweets
Here you have your first 10 tweets
Adding  1000 to the dataframe
Your dataframe has a total of 0 tweets.
Oldest tweet is from nan


<ipython-input-13-9fe91170165c>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tw = df_tw.append(df2, sort=True)


,date,id,query,tweet_text,user
